In [ ]:
# coding:utf8


## 【1】协程获取字典list

In [11]:
import asyncio

In [1]:
import multiprocessing 
mpools = multiprocessing.Pool(3)

In [ ]:
import threading
mpools = threadin

### 关键的多进程mp执行函数

In [4]:
import multiprocessing 
def mp(fun,*args,**kwargs):
    def _fun(fun,q):
        print(f"正在启用进程执行函数：【 {fun.__name__} 】")
        res = fun(*args,**kwargs)
        q.put(res)
        return
    q1 = multiprocessing.Queue(1)
    p1 = multiprocessing.Process(target=_fun,args=(fun,q1))
    p1.start()
    res = q1.get()
    p1.join()
    return res


In [5]:
mp(print,1)

正在启用进程执行函数：【 print 】
1


In [6]:
from search_main import get_res_dicts

In [44]:
res = mp(get_res_dicts)

正在启用进程执行函数：【 get_res_dicts 】
将要POST：【 http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=1&type=shj 】
将要POST：【 http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=2&type=shj 】
将要POST：【 http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=3&type=shj 】
将要POST：【 http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=4&type=shj 】
将要POST：【 http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=5&type=shj 】
将要POST：【 http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=6&type=shj 】

In [45]:
dicts_list = res

In [8]:
len(res)

209

In [28]:
res

[]

### 裁剪结果

In [9]:
cut_res = res[0:50]

In [22]:
len(cut_res)

50

In [46]:
cut_res

[{'secCode': '000039',
  'secName': '中集集团',
  'orgId': 'gssz0000039',
  'type': 'shj',
  'announcementId': '1204526484',
  'announcementTitle': '中集集团：2017年<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告',
  'announcement_title': '中集集团：2017年社会责任暨环境、社会及管治报告',
  'announcementTime': 1522166400000,
  'announcement_year': '2018',
  'adjunctUrl': 'finalpage/2018-03-28/1204526484.PDF',
  'record_time': '2023-01-31'},
 {'secCode': '000039',
  'secName': '中集集团',
  'orgId': 'gssz0000039',
  'type': 'shj',
  'announcementId': '1205949450',
  'announcementTitle': '中集集团：2018<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告',
  'announcement_title': '中集集团：2018社会责任暨环境、社会及管治报告',
  'announcementTime': 1553702400000,
  'announcement_year': '2019',
  'adjunctUrl': 'finalpage/2019-03-28/1205949450.PDF',
  'record_time': '2023-01-31'},
 {'secCode': '000039',
  'secName': '中集集团',
  'orgId': 'gssz0000039',
  'type': 'shj',
  'announcementId': '1207411179',
  'announcementTitle': '中集集团：<em>社会</em>责任暨环境、<

## 【2】执行函数

In [47]:
from search_main import *

In [102]:
SEARCH_RES_NAME = "asin_spark"

### 源代码

In [ ]:
dirs = download_from_dicts(res_dicts,f'./data/search_res/{SEARCH_RES_NAME}')

# multiprocess
import multiprocessing

# 使用多进程分析pdf列表，返回结果字典列表（每个字典对应一个pdf的分析结果）
value = multiprocessing.Manager().Value('i',0)
lock = multiprocessing.Manager().Lock()
total_num = len(dirs)
args_list = list(zip(dirs,[value]*total_num,[total_num]*total_num,[lock]*total_num))
parse_res = multi_process_star(parse_pdf_mp,args_list)
# 用pdf列表的分析结果更新res_dicts
[res_dict.update(pdf_res_dict) for pdf_res_dict,res_dict in zip(parse_res,res_dicts)]

# 确保写入的一致性
assert list(res_dicts[0].keys()) == list(TABLE_INFO.keys())


db = quickDb.MyDb('Juchao')
db.execute(quickDb.generate_create_sql(table_name=TABLE_NAME_search,columns_infos_dict=TABLE_INFO,primary_key=TABLE_PRIMARY))
# 写入announcements结果
db.execute(quickDb.generate_insert_sql(table_name=TABLE_NAME_search,input_args=res_dicts))
db.commit()
db.close()

### 最终函数

## 【3】pyspark

### 初始化

In [27]:
import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = "--num-executors 10"
# os.unsetenv('PYSPARK_SUBMIT_ARGS')
#spark.driver.extraJavaOptions=-Dhttp.proxyHost=proxy.mycorp.com-Dhttp.proxyPort=1234 -Dhttp.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 -Dhttps.proxyHost=proxy.mycorp.com -Dhttps.proxyPort=1234 -Dhttps.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 pyspark-shell"

In [19]:
from pyspark import SparkConf,SparkContext

In [28]:
conf = SparkConf().setMaster("yarn").setAppName("Hello").set("spark.executor.instances",'4').set("spark.executor.cores",'2')

In [29]:
sc = SparkContext(conf=conf)


In [4]:
import pandas

ModuleNotFoundError: No module named 'pandas'

### 关闭sc

In [26]:
sc.stop()

### 测试

#### split

In [33]:
rdd = sc.parallelize(cut_res,8)
#.map(lambda x:x+100)


In [34]:
split_res = rdd.glom().collect()

In [35]:
for i in split_res:
    print(len(i))

6
6
6
6
6
6
6
8


#### map字典列表处理

In [115]:
import asyncio
from socket import timeout
import aiohttp
import os
import time
import random

import config
from config import requests_config
HEADER = requests_config.HEADER
PORT = requests_config.PORT
PORT=-1
if PORT==-1:
    PROXIES=''
else:
    PROXIES = f"http://127.0.0.1:{PORT}"


COUNT_LIST = [0]
TOTAL_LIST = [1]
TIMEOUT_TAO = 5 # 二进制指数退避的争用期窗口大小
TIMEOUT = 5
def timeit(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"{func.__name__} took 【 {end - start:.6f} 】seconds.")
        return res

    return wrapper


def random_time_out(timeout_times,):
    if timeout_times > 0:
        time.sleep(TIMEOUT_TAO * random.uniform(0,pow(2,timeout_times)-1))
    timeout_times += 1
    return timeout_times


def get_progress(count,total):
    """[012/100]"""
    max_len = len(str(total))
    count_len = len(str(count))
    count_res = '0'*(max_len-count_len)+str(count)
    return f"[{count_res}/{total}]"



async def get_json(session, url,):
    """
    :return: response.json()
    """ 
    print(f"将要POST：【 {url} 】")
    timeout_times = 0
    while True:
        timeout_times = random_time_out(timeout_times,)
        try:
            response = await session.get(url,headers=HEADER,proxy=PROXIES,timeout=TIMEOUT)
            if response.status == 200:
                break
        except Exception as e:
            print(e)
            continue
        
    res = await response.json()
    COUNT_LIST[0] += 1
    print(f"{get_progress(COUNT_LIST[0],TOTAL_LIST[0])} √ POST")
    return res



async def post_json(session, url, data,):
    """
    :return: response.json()
    """ 
    print(f"将要POST：【 {url} 】")
    timeout_times = 0
    while True:
        timeout_times = random_time_out(timeout_times,)
        try:
            response = await session.post(url, data=data, headers=HEADER,proxy=PROXIES,timeout=TIMEOUT)
            if response.status == 200:
                break
        except Exception as e:
            print(e)
            continue
        
    res = await response.json()
    COUNT_LIST[0] += 1
    print(f"{get_progress(COUNT_LIST[0],TOTAL_LIST[0])} √ POST")
    return res


async def download(session, name_url, to_dir):
    """返回response.text

    """
    try:
        # os.mkdir(to_dir)
        os.makedirs(to_dir)
        print(f"已创建文件夹{to_dir}")
    except:
        pass

    name,url = name_url
    print(f"将要下载：【 {name} | {url} 】")
    
    file_path = os.path.join(to_dir,name)

    # time.sleep(random.uniform(1,2)/5)
    timeout_times = 0
    file_size_bytes = 0
    # timeout = TIMEOUT
    while True:
        timeout_times = random_time_out(timeout_times,)
        try:
            response = await session.get(url, headers=HEADER,proxy=PROXIES,)
            file_size_bytes = int(response.headers.get('Content-Length'))
            
            if response.status == 200:
                pass
            else:
                raise asyncio.exceptions.TimeoutError
            
            # 若已下载则跳过
            if os.path.isfile(file_path):
                with open(file_path,'rb')as f:
                    if len(f.read()) == file_size_bytes:
                        COUNT_LIST[0] += 1
                        print(f"{get_progress(COUNT_LIST[0],TOTAL_LIST[0])}  ! 文件已存在：【 {name} 】")
                        return file_path,0
                    else:
                        print(f"× 文件大小异常，准备重新下载： {name} ")

            # 写入
            with open(file_path,'wb')as f:
                f.write(await response.content.read())
                COUNT_LIST[0] += 1
                print(f"{get_progress(COUNT_LIST[0],TOTAL_LIST[0])}  √ 下载完成：【 {name} 】")

            break

        except asyncio.exceptions.TimeoutError:
            print(f"第 {timeout_times} 次超时...文件为：【 {name} 】")
            continue
        except Exception as e:
            print(e)
            if timeout_times < 5:
                continue
            else:
                break


    return file_path,file_size_bytes/1024/1024


def downloads_tasks(session, names_urls,to_dir):
    if type(to_dir)==str:
        tasks = [asyncio.create_task(download(session, name_url,to_dir)) for name_url in names_urls]
    elif type(to_dir)==list:
        tasks = [asyncio.create_task(download(session, name_url,to_dir0)) for (name_url,to_dir0) in zip(names_urls,to_dir) ]
    # print(tasks)
    return tasks
@timeit
def async_downloads(names_urls,to_dir):
    async def _async_downloads(names_urls,to_dir):
        # import getUrls
        # names_urls = getUrls.get_name_url(stack_code, START_DATE, END_DATE,**args)
        start = time.perf_counter()
        COUNT_LIST[0] = 0
        TOTAL_LIST[0] = len(names_urls)
        async with aiohttp.ClientSession() as session:
            tasks = downloads_tasks(session,names_urls,to_dir)
            res = await asyncio.gather(*tasks)    
        download_dir,download_sizes = list(zip(*res))
        end = time.perf_counter()
        all_size = sum(download_sizes)
        spend_time = end - start
        print(f"共下载 【 {all_size:.3f} 】 MB 数据")
        print(f"共花费 【 {spend_time:.3f} 】 秒")
        print(f"下载速度为 【 {all_size/spend_time:.3f} 】 MB/s")
        return download_dir
    return asyncio.run(_async_downloads(names_urls,to_dir))

def download_from_dicts(res_dicts,to_dir):
    """将传入的res_dicts中的每一项提取其中的pdf链接进行下载,返回下载完成后对应的文件路径(列表)"""
    pdf_urls = [(dict0['announcement_title']+'.pdf','http://static.cninfo.com.cn/'+dict0['adjunctUrl']) for dict0 in res_dicts]
    # 使用协程下载pdf文件，并返回所有文件下载到的路径（类型：tuple）
    dirs = async_downloads(pdf_urls,to_dir)
    return dirs


In [112]:
def do_to_dicts(dicts_list):
    return download_from_dicts(dicts_list,f'/root/export/data/{SEARCH_RES_NAME}')
    return [i['announcement_title'] for i in  dicts_list]

In [69]:
do_to_dicts(cut_res)

['中集集团：2017年社会责任暨环境、社会及管治报告',
 '中集集团：2018社会责任暨环境、社会及管治报告',
 '中集集团：社会责任暨环境、社会及管治报告（2019）',
 '中集集团：2016年环境、社会及管治报告',
 '申万宏源：2018年度环境、社会及管治报告',
 '珠海港：珠海港股份有限公司2020年度环境、社会及管治（ESG）报告',
 '珠海港：2021年度环境、社会及管治（ESG）报告',
 '丽珠集团：2020年度环境、社会及管治报告',
 '丽珠集团：2021年度环境、社会及管治报告',
 '丽珠集团：2017年度环境、社会及管治报告',
 '丽珠集团：2016年度环境、社会及管治报告',
 '丽珠集团：2018年度环境、社会及管治报告',
 '丽珠集团：2019年环境、社会及管治报告',
 '*ST东电：2018环境、社会及管治报告',
 '长春高新：2021年度环境、社会及管治（ESG）报告',
 '美锦能源：2021年环境、社会及管治报告',
 '北新建材：2021年度环境、社会及管治（ESG）暨社会责任报告',
 '顺鑫农业：2021年度环境、社会及管治（ESG）报告',
 '天山股份：新疆天山水泥股份有限公司2021年度环境、社会及管治（ESG）报告',
 '海信家电：2018年度环境、社会及管治报告',
 '海信家电：2020年度环境、社会及管治报告',
 '海信家电：2021年度环境、社会及管治报告',
 '海信家电：2019年度环境、社会及管治报告',
 '海信科龙：2017年度环境、社会及管治报告',
 '中南建设：2021年环境、社会及管治报告（更新）',
 '中南建设：关于2021年环境、社会及管治报告的更正公告',
 '越秀金控：2021年环境、社会及管治（ESG）报告',
 '电投能源：2021年度环境、社会及管治（ESG）报告',
 '纳思达：2021环境、社会及管治报告(中文版)',
 '大华股份：2021年大华股份环境、社会及管治报告',
 '九阳股份：2020年环境、社会及管治报告',
 '九阳股份：2021年环境、社会及管治报告',
 '广联达：2021年环境、社会及管治（ESG）报告',
 '海康威视：2021年环境、社会及管治报告',
 '海康威视：2020年环境、社会及管治报告',
 '协鑫集成

#### map

In [37]:
rdd0 = sc.parallelize(list(range(100)),8)

In [38]:
rdd1 = rdd0.map(lambda x:x%5)

In [41]:
rdd2 = rdd1.map(lambda x:x+100)


In [42]:
res = rdd2.collect()

In [43]:
res

[100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104,
 100,
 101,
 102,
 103,
 104]

#### map1

In [104]:
rdd0 = sc.parallelize(dicts_list,8)
#.map(lambda x:x+100)

In [105]:
rdd1 = rdd0.glom()

In [106]:
res = rdd1.collect()

In [107]:
len(res)

8

In [108]:
len(res[0])

26

In [91]:
res[0]

[{'secCode': '000039',
  'secName': '中集集团',
  'orgId': 'gssz0000039',
  'type': 'shj',
  'announcementId': '1204526484',
  'announcementTitle': '中集集团：2017年<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告',
  'announcement_title': '中集集团：2017年社会责任暨环境、社会及管治报告',
  'announcementTime': 1522166400000,
  'announcement_year': '2018',
  'adjunctUrl': 'finalpage/2018-03-28/1204526484.PDF',
  'record_time': '2023-01-31'},
 {'secCode': '000039',
  'secName': '中集集团',
  'orgId': 'gssz0000039',
  'type': 'shj',
  'announcementId': '1205949450',
  'announcementTitle': '中集集团：2018<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告',
  'announcement_title': '中集集团：2018社会责任暨环境、社会及管治报告',
  'announcementTime': 1553702400000,
  'announcement_year': '2019',
  'adjunctUrl': 'finalpage/2019-03-28/1205949450.PDF',
  'record_time': '2023-01-31'},
 {'secCode': '000039',
  'secName': '中集集团',
  'orgId': 'gssz0000039',
  'type': 'shj',
  'announcementId': '1203671603',
  'announcementTitle': '中集集团：2016年环境、<em>社会</em

In [90]:
res

[[{'secCode': '000039',
   'secName': '中集集团',
   'orgId': 'gssz0000039',
   'type': 'shj',
   'announcementId': '1204526484',
   'announcementTitle': '中集集团：2017年<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告',
   'announcement_title': '中集集团：2017年社会责任暨环境、社会及管治报告',
   'announcementTime': 1522166400000,
   'announcement_year': '2018',
   'adjunctUrl': 'finalpage/2018-03-28/1204526484.PDF',
   'record_time': '2023-01-31'},
  {'secCode': '000039',
   'secName': '中集集团',
   'orgId': 'gssz0000039',
   'type': 'shj',
   'announcementId': '1205949450',
   'announcementTitle': '中集集团：2018<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告',
   'announcement_title': '中集集团：2018社会责任暨环境、社会及管治报告',
   'announcementTime': 1553702400000,
   'announcement_year': '2019',
   'adjunctUrl': 'finalpage/2019-03-28/1205949450.PDF',
   'record_time': '2023-01-31'},
  {'secCode': '000039',
   'secName': '中集集团',
   'orgId': 'gssz0000039',
   'type': 'shj',
   'announcementId': '1203671603',
   'announcementTitl

In [109]:
rdd3 = sc.parallelize(res,4)

In [95]:
len(rdd3.glom().collect()[1])

2

In [96]:
for i in rdd3.collect():
    print(len(i))

26
26
26
26
26
26
26
27


In [116]:
rdd4 = rdd3.map(do_to_dicts)

In [117]:
res4 = rdd4.collect()

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 4) / 4]
Traceback (most recent call last):
  File "/home/arcsinszy/App/anaconda/anaconda3/envs/pyspark/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/arcsinszy/App/anaconda/anaconda3/envs/pyspark/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/arcsinszy/App/anaconda/anaconda3/envs/pyspark/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [99]:
len(res4)

8

In [100]:
rdd3.collect()[0]

[{'secCode': '000039',
  'secName': '中集集团',
  'orgId': 'gssz0000039',
  'type': 'shj',
  'announcementId': '1204526484',
  'announcementTitle': '中集集团：2017年<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告',
  'announcement_title': '中集集团：2017年社会责任暨环境、社会及管治报告',
  'announcementTime': 1522166400000,
  'announcement_year': '2018',
  'adjunctUrl': 'finalpage/2018-03-28/1204526484.PDF',
  'record_time': '2023-01-31'},
 {'secCode': '000039',
  'secName': '中集集团',
  'orgId': 'gssz0000039',
  'type': 'shj',
  'announcementId': '1205949450',
  'announcementTitle': '中集集团：2018<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告',
  'announcement_title': '中集集团：2018社会责任暨环境、社会及管治报告',
  'announcementTime': 1553702400000,
  'announcement_year': '2019',
  'adjunctUrl': 'finalpage/2019-03-28/1205949450.PDF',
  'record_time': '2023-01-31'},
 {'secCode': '000039',
  'secName': '中集集团',
  'orgId': 'gssz0000039',
  'type': 'shj',
  'announcementId': '1203671603',
  'announcementTitle': '中集集团：2016年环境、<em>社会</em

In [ ]:
rdd0 = sc.parallelize(dicts_list,8)